# Compare NLP libraries

In [95]:
# -*- coding: utf-8 -*-
# 分词
from preprocessing import preprocess_string
from preprocessing import strip_numeric
from preprocessing import remove_stopwords
from preprocessing import strip_punctuation
from preprocessing import tokenize
from snownlp import SnowNLP
from pyltp import SentenceSplitter, Segmentor, Postagger, Parser, NamedEntityRecognizer, SementicRoleLabeller
import codecs
from timeit import default_timer
import time
import os
from jieba.analyse import extract_tags, textrank
user_path = os.path.expanduser("~")
from nltk.tag import StanfordNERTagger
from nltk.tag import StanfordPOSTagger
from nltk.parse.stanford import StanfordParser



## Tokenize

### jieba cut

In [9]:
import jieba
jieba.load_userdict(user_path + "/share/deep_learning/data/dict/jieba.txt")
jieba.analyse.set_stop_words(
    user_path + "/projects/deep_learning/knowledge_graph/information/stopwords")
begin = default_timer()


In [2]:
str_test = u'''云南铜业股份有限公司（深交所：000878），简称云铜股份、云铜，
前身为云南冶炼厂，成立于1958年，1998年改制为股份公司，更名为现称，
1998年6月2日于深圳证券交易所上市。公司是中国第四大铜业企业，生产高纯阴极铜、
电工用铜线坏、工业硫酸、金锭、银锭、电工用圆铜线、硫酸铜等主产品，
并能综合回收金、银、铝、铋、铂、钯等多种有色金属。2007年10月，
中国铝业收购云铜母公司云南铜业集团的49%股权，改名“中铝云南铜业集团”。'''


In [7]:
filter_setting = [tokenize, strip_punctuation]
text = preprocess_string(str_test, filter_setting)
print(text)

云南铜业 股份有限公司   深交所   000878     简称 云铜 股份   云铜   
 前身 为 云南 冶炼厂   成立 于 1958 年   1998 年 改制 为 股份公司   更名 为 现称   
 1998 年 6 月 2 日于 深圳证券交易所 上市   公司 是 中国 第四 大 铜业 企业   生产 高纯 阴极铜   
 电工 用 铜线 坏   工业 硫酸   金锭   银锭   电工 用圆 铜线   硫酸铜 等 主产品   
 并 能 综合 回收 金   银   铝   铋   铂   钯 等 多种 有色金属   2007 年 10 月   
 中国铝业 收购 云铜 母公司 云南铜业 集团 的 49  股权   改名   中铝 云南铜业 集团    


### SnowNLP

In [45]:
filter_setting = [strip_punctuation]
text = preprocess_string(str_test, filter_setting)
s = SnowNLP(text)
print('\t'.join(s.words))

云南	铜业	股份	有限公司	深交所	000878	简称	云	铜	股份	云	铜	前身	为	云南	冶炼厂	成立	于	1958	年	1998	年	改制	为	股份公司	更名	为	现	称	1998	年	6	月	2	日于	深圳	证券	交易所	上市	公司	是	中国	第四	大	铜业	企业	生产	高	纯	阴极	铜	电工	用	铜线	坏	工业	硫酸	金锭	银锭	电工	用	圆	铜线	硫酸	铜	等	主产品	并	能	综合	回收	金	银	铝	铋	铂	钯	等	多种	有色金属	2007	年	10	月	中国	铝业	收购	云	铜	母公司	云南	铜业	集团	的	49	股权	改名	中	铝	云南	铜业	集团


### 哈工大pyltp

In [ ]:
LTP_DATA_DIR = '/home/weiwu/share/software/ltp_data_v3.4.0'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR,
                              'cws.model')  # 分词模型路径，模型名称为`cws.model`

In [36]:
from pyltp import Segmentor
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型
words = segmentor.segment(text)  # 分词
print('\t'.join(words))
# segmentor.release()  # 释放模型

云南	铜业	股份	有限公司	深交所000878	简称	云铜	股份	云	铜
	前身	为	云南	冶炼厂	成立	于	1958年	1998年	改制	为	股份公司	更名	为	现	称	
	1998年	6月	2日	于	深圳	证券	交易所	上市	公司	是	中国	第四	大	铜业	企业	生产	高	纯阴	极	铜
	电工	用	铜线	坏	工业	硫酸金锭	银锭	电工	用	圆	铜线	硫酸铜	等	主产品	
	并	能	综合	回收	金银	铝铋	铂钯	等	多种	有色金属	2007年	10月	
	中国	铝业	收购	云	铜母	公司	云南	铜业	集团	的	49	股权	改名	中	铝	云南	铜业	集团


## Keywords

In [19]:
print("TF-IDF")
for keyword, weight in extract_tags(str_test, topK=10, withWeight=True):
    print('%s %s' % (keyword, weight))

print('\n')
print("Text Rank")
for keyword, weight in textrank(str_test, withWeight=True):
    print('%s %s' % (keyword, weight))


TF-IDF
云铜 0.6078695340457627
云南铜业 0.4994817054950848
1998 0.40524635603050846
铜线 0.38426197608474577
电工 0.2911007346755932
阴极铜 0.2238564486677966
金锭 0.21210819137118644
股份有限公司 0.20262317801525423
000878 0.20262317801525423
1958 0.20262317801525423


Text Rank
铜线 1.0
集团 0.8588945863309484
电工 0.8019967387429742
中国 0.7871941176415925
企业 0.7286800692243445
股份公司 0.7089238629634947
更名 0.7038034747106456
金锭 0.6481616666859029
硫酸 0.6478215631892147
母公司 0.6337320036376739
成立 0.5950407383943361
股份 0.594869945137106
综合 0.594869945137106
冶炼厂 0.5922642962224522
简称 0.590517768532632
回收 0.590517768532632
云南 0.5891731073304121
股权 0.5667583077673782
铜业 0.5660237527952675
改名 0.5638114334139906


### SnowNLP

In [41]:
print('keywords', s.keywords(10)) 

keywords ['铜业', '铜', '年', '云南', '铝', '中国', '云', '1998', '月', '硫酸']


## 词性标注


### pyltp

In [51]:
pos_model_path = os.path.join(LTP_DATA_DIR,
                              'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
postagger = Postagger()  # 初始化实例
postagger.load(pos_model_path)  # 加载模型

In [52]:
postags = postagger.postag(words)  # 词性标注

In [57]:
print(list(zip(segmentor.segment(text),postags)))

[('云南', 'ns'), ('铜业', 'n'), ('股份', 'n'), ('有限公司', 'n'), ('深交所000878', 'nz'), ('简称', 'v'), ('云铜', 'nz'), ('股份', 'n'), ('云', 'n'), ('铜\n', 'n'), ('前身', 'n'), ('为', 'v'), ('云南', 'ns'), ('冶炼厂', 'n'), ('成立', 'v'), ('于', 'p'), ('1958年', 'nt'), ('1998年', 'nt'), ('改制', 'v'), ('为', 'v'), ('股份公司', 'i'), ('更名', 'v'), ('为', 'v'), ('现', 'nt'), ('称', 'v'), ('\n', 'v'), ('1998年', 'nt'), ('6月', 'nt'), ('2日', 'nt'), ('于', 'p'), ('深圳', 'ns'), ('证券', 'n'), ('交易所', 'n'), ('上市', 'v'), ('公司', 'n'), ('是', 'v'), ('中国', 'ns'), ('第四', 'm'), ('大', 'a'), ('铜业', 'n'), ('企业', 'n'), ('生产', 'v'), ('高', 'a'), ('纯阴', 'n'), ('极', 'd'), ('铜\n', 'a'), ('电工', 'n'), ('用', 'p'), ('铜线', 'n'), ('坏', 'a'), ('工业', 'n'), ('硫酸金锭', 'n'), ('银锭', 'n'), ('电工', 'n'), ('用', 'p'), ('圆', 'a'), ('铜线', 'n'), ('硫酸铜', 'n'), ('等', 'u'), ('主产品', 'n'), ('\n', 'n'), ('并', 'c'), ('能', 'v'), ('综合', 'v'), ('回收', 'v'), ('金银', 'b'), ('铝铋', 'n'), ('铂钯', 'n'), ('等', 'u'), ('多种', 'm'), ('有色金属', 'n'), ('2007年', 'nt'), ('10月', 'nt'), ('\n', 'v'), ('中国', 'n

### SnowNLP

In [56]:
print(list(s.tags))

[('云南', 'ns'), ('铜业', 'nz'), ('股份', 'n'), ('有限公司', 'n'), ('深交所', 'j'), ('000878', 'j'), ('简称', 'v'), ('云', 'j'), ('铜', 'j'), ('股份', 'n'), ('云', 'j'), ('铜', 'j'), ('前身', 'n'), ('为', 'p'), ('云南', 'ns'), ('冶炼厂', 'n'), ('成立', 'v'), ('于', 'p'), ('1958', 'm'), ('年', 'q'), ('1998', 'm'), ('年', 'q'), ('改制', 'v'), ('为', 'p'), ('股份公司', 'l'), ('更名', 'v'), ('为', 'p'), ('现', 'Tg'), ('称', 'v'), ('1998', 'Tg'), ('年', 'q'), ('6', 'ns'), ('月', 'n'), ('2', 'Rg'), ('日于', 'Rg'), ('深圳', 'ns'), ('证券', 'n'), ('交易所', 'n'), ('上市', 'vn'), ('公司', 'n'), ('是', 'v'), ('中国', 'ns'), ('第四', 'm'), ('大', 'a'), ('铜业', 'u'), ('企业', 'n'), ('生产', 'v'), ('高', 'a'), ('纯', 'a'), ('阴极', 'u'), ('铜', 'n'), ('电工', 'n'), ('用', 'v'), ('铜线', 'u'), ('坏', 'a'), ('工业', 'n'), ('硫酸', 'Rg'), ('金锭', 'Rg'), ('银锭', 'Rg'), ('电工', 'n'), ('用', 'v'), ('圆', 'Vg'), ('铜线', 'Ag'), ('硫酸', 'Bg'), ('铜', 'n'), ('等', 'u'), ('主产品', 'an'), ('并', 'c'), ('能', 'v'), ('综合', 'vn'), ('回收', 'vn'), ('金', 'nr'), ('银', 'nr'), ('铝', 'o'), ('铋', 'o'), ('铂', 'e'), ('钯',

### CoreNLP

In [70]:
# StanfordPOSTagger 中文词性标注
pos_tagger_model_file_path = '/home/weiwu/share/software/chinese-distsim.tagger'
pos_tagger_jar_file_path = '/home/weiwu/tools/stanford-postagger-full-2017-06-09/stanford-postagger-3.8.0.jar'

ch_pos_tagger = StanfordPOSTagger(
    model_filename=pos_tagger_model_file_path,
    path_to_jar=pos_tagger_jar_file_path)
pos_result = ch_pos_tagger.tag(words)


/home/weiwu/.virtualenvs/graph/lib/python3.6/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [72]:
print(pos_result)

[('', '云南#NR'), ('', '铜业#NR'), ('', '股份#NN'), ('', '有限公司#VV'), ('', '深交所000878#AD'), ('', '简称#VV'), ('', '云铜#NR'), ('', '股份#NN'), ('', '云#NN'), ('', '铜#NN'), ('', '前身#NN'), ('', '为#VC'), ('', '云南#NR'), ('', '冶炼厂#NN'), ('', '成立#VV'), ('', '于#P'), ('', '1958年#NT'), ('', '1998年#NT'), ('', '改制#VV'), ('', '为#VC'), ('', '股份公司#NN'), ('', '更名#VV'), ('', '为#VC'), ('', '现#AD'), ('', '称#VV'), ('', '1998年#NT'), ('', '6月#NT'), ('', '2日#NT'), ('', '于#P'), ('', '深圳#NR'), ('', '证券#NN'), ('', '交易所#NN'), ('', '上市#NN'), ('', '公司#NN'), ('', '是#VC'), ('', '中国#NR'), ('', '第四#OD'), ('', '大#JJ'), ('', '铜业#NN'), ('', '企业#NN'), ('', '生产#NN'), ('', '高#VA'), ('', '纯阴#VA'), ('', '极#AD'), ('', '铜#NN'), ('', '电工#NN'), ('', '用#P'), ('', '铜线#NN'), ('', '坏#JJ'), ('', '工业#NN'), ('', '硫酸金锭#VV'), ('', '银锭#NN'), ('', '电工#NN'), ('', '用#P'), ('', '圆#NN'), ('', '铜线#NN'), ('', '硫酸铜#NR'), ('', '等#ETC'), ('', '主产品#NN'), ('', '并#AD'), ('', '能#VV'), ('', '综合#VV'), ('', '回收#NN'), ('', '金银#NN'), ('', '铝铋#NN'), ('', '铂钯#NN'), ('', '等

## 命名实体识别(NER)

### pyltp

In [54]:
ner_model_path = os.path.join(LTP_DATA_DIR,
                              'ner.model')  # 命名实体识别模型路径，模型名称为`pos.model`


In [55]:
recognizer = NamedEntityRecognizer()  # 初始化实例
recognizer.load(ner_model_path)  # 加载模型

netags = recognizer.recognize(words, postags)  # 命名实体识别

#print('\t'.join(netags))
print(list(zip(words, netags)))

[('云南', 'B-Ni'), ('铜业', 'I-Ni'), ('股份', 'I-Ni'), ('有限公司', 'E-Ni'), ('深交所000878', 'O'), ('简称', 'O'), ('云铜', 'O'), ('股份', 'O'), ('云', 'O'), ('铜\n', 'O'), ('前身', 'O'), ('为', 'O'), ('云南', 'S-Ns'), ('冶炼厂', 'O'), ('成立', 'O'), ('于', 'O'), ('1958年', 'O'), ('1998年', 'O'), ('改制', 'O'), ('为', 'O'), ('股份公司', 'O'), ('更名', 'O'), ('为', 'O'), ('现', 'O'), ('称', 'O'), ('\n', 'O'), ('1998年', 'O'), ('6月', 'O'), ('2日', 'O'), ('于', 'O'), ('深圳', 'B-Ni'), ('证券', 'I-Ni'), ('交易所', 'I-Ni'), ('上市', 'I-Ni'), ('公司', 'E-Ni'), ('是', 'O'), ('中国', 'S-Ns'), ('第四', 'O'), ('大', 'O'), ('铜业', 'O'), ('企业', 'O'), ('生产', 'O'), ('高', 'O'), ('纯阴', 'O'), ('极', 'O'), ('铜\n', 'O'), ('电工', 'O'), ('用', 'O'), ('铜线', 'O'), ('坏', 'O'), ('工业', 'O'), ('硫酸金锭', 'O'), ('银锭', 'O'), ('电工', 'O'), ('用', 'O'), ('圆', 'O'), ('铜线', 'O'), ('硫酸铜', 'O'), ('等', 'O'), ('主产品', 'O'), ('\n', 'O'), ('并', 'O'), ('能', 'O'), ('综合', 'O'), ('回收', 'O'), ('金银', 'O'), ('铝铋', 'O'), ('铂钯', 'O'), ('等', 'O'), ('多种', 'O'), ('有色金属', 'O'), ('2007年', 'O'), ('10月', 'O'), ('\

### Stanford CoreNLP

In [74]:
begin = default_timer()
en_model_file_path = '/home/weiwu/tools/stanford-ner-2017-06-09/classifiers/english.all.3class.distsim.crf.ser.gz'
ch_model_file_path = '/home/weiwu/share/software/chinese.misc.distsim.crf.ser.gz'
jar_path = '/home/weiwu/tools/stanford-ner-2017-06-09/stanford-ner-3.8.0.jar'
# StanfordNERTagger 中文命名实体识别
ch_tagger = StanfordNERTagger(
    model_filename=ch_model_file_path, path_to_jar=jar_path)
end = default_timer()
result = ch_tagger.tag(words)

load_duration = end - begin
print("Total procesing time: %.1fs seconds" % (end - begin))


/home/weiwu/.virtualenvs/graph/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


Total procesing time: 0.0s seconds


In [77]:
stanford_ner = []
for word, tag in result:
    stanford_ner.append((word, tag))

In [78]:
print(stanford_ner)

[('云南', 'ORGANIZATION'), ('铜业', 'ORGANIZATION'), ('股份', 'ORGANIZATION'), ('有限公司', 'ORGANIZATION'), ('深交所000878', 'ORGANIZATION'), ('简称', 'ORGANIZATION'), ('云铜', 'ORGANIZATION'), ('股份', 'ORGANIZATION'), ('云', 'O'), ('铜', 'O'), ('前身', 'O'), ('为', 'O'), ('云南', 'ORGANIZATION'), ('冶炼厂', 'ORGANIZATION'), ('成立', 'O'), ('于', 'O'), ('1958年', 'MISC'), ('1998年', 'MISC'), ('改制', 'O'), ('为', 'O'), ('股份公司', 'O'), ('更名', 'O'), ('为', 'O'), ('现', 'O'), ('称', 'O'), ('1998年', 'MISC'), ('6月', 'MISC'), ('2日', 'MISC'), ('于', 'O'), ('深圳', 'ORGANIZATION'), ('证券', 'ORGANIZATION'), ('交易所', 'ORGANIZATION'), ('上市', 'O'), ('公司', 'O'), ('是', 'O'), ('中国', 'GPE'), ('第四', 'MISC'), ('大', 'O'), ('铜业', 'O'), ('企业', 'O'), ('生产', 'O'), ('高', 'O'), ('纯阴', 'O'), ('极', 'O'), ('铜', 'O'), ('电工', 'O'), ('用', 'O'), ('铜线', 'O'), ('坏', 'O'), ('工业', 'O'), ('硫酸金锭', 'ORGANIZATION'), ('银锭', 'ORGANIZATION'), ('电工', 'ORGANIZATION'), ('用', 'O'), ('圆', 'O'), ('铜线', 'O'), ('硫酸铜', 'O'), ('等', 'O'), ('主产品', 'O'), ('并', 'O'), ('能', 'O'), ('综合'

In [ ]:
print(load_duration)

## 句法分析

### pyltp

In [59]:
parser = Parser()
parser.load(os.path.join(LTP_DATA_DIR, "parser.model"))
arcs = parser.parse(words, postags)

print("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))

2:ATT	4:ATT	4:ATT	6:SBV	6:SBV	0:HED	9:ATT	9:ATT	10:ATT	11:ATT	6:VOB	15:ADV	14:ATT	12:POB	6:VOB	15:CMP	18:ATT	16:POB	25:SBV	25:ADV	20:POB	25:SBV	25:ADV	25:ADV	15:COO	36:SBV	28:ATT	29:ATT	36:ADV	36:ADV	33:ATT	33:ATT	30:POB	35:ATT	30:POB	25:VOB	40:ATT	39:ATT	40:ATT	41:ATT	44:ATT	44:ATT	44:ATT	47:ATT	46:ADV	47:ATT	65:SBV	65:ADV	52:ATT	51:ATT	52:ATT	53:ATT	54:ATT	48:POB	65:ADV	57:ATT	58:ATT	60:ATT	58:RAD	61:ATT	55:POB	65:ADV	65:ADV	65:ADV	36:VOB	67:ATT	68:ATT	71:ATT	68:RAD	71:ATT	65:VOB	73:ATT	74:ADV	65:COO	76:ATT	80:ATT	80:ATT	79:ATT	80:ATT	83:ATT	83:ATT	83:ATT	86:ATT	83:RAD	86:ATT	87:SBV	88:ATT	89:ATT	92:ATT	91:ATT	92:ATT	74:VOB


### StanfordParser 中文句法分析

In [98]:
begin = time.time()
path_to_jar=r'/home/weiwu/share/software/stanford-parser.jar'
path_to_models_jar=r'/home/weiwu/share/software/stanford-parser-3.8.0-models.jar'
model_path = r'/home/weiwu/share/software/chinesePCFG.ser.gz'
ch_parser = StanfordParser(path_to_jar, path_to_models_jar, model_path)

end =  time.time()
load_duration = end - begin
print(load_duration)

2.3296425342559814


### To-fix

In [114]:
ch_parser.parse(u'俄罗斯 希望 伊朗 没有 制造 核武器 计划')

OSError: [Errno 7] Argument list too long: '/usr/bin/java'

## 语义角色标注(Semantic Role Labeling)

In [61]:
labeller = SementicRoleLabeller()
labeller.load(os.path.join(LTP_DATA_DIR, "pisrl.model"))

In [62]:
roles = labeller.label(words, postags, arcs)

for role in roles:
    print(role.index, "".join(
            ["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments]))

5 A1:(0,3)
21 A2:(25,25)
35 A0:(33,34)
63 A1:(65,68)
64 MNR:(47,53)MNR:(54,60)A1:(65,68)
86 A1:(74,85)
